# Analisis de superficial del csv de clicks
### En este notebook exploramos los registros del csv con el objetivo de buscar relaciones entre las distintas variables

In [1]:
import pandas as pd
import numpy as np

In [2]:
auctions = pd.read_csv("../data/auctions.csv", dtype={'date': 'str','device_id': 'category', 'ref_type_id': 'category','source_id':'category'}, parse_dates = ['date'])
auctions.drop('ref_type_id', axis=1, inplace = True)
auctions.drop('source_id', axis=1, inplace = True)
auctions = auctions.rename(columns = {'device_id':'ref_hash'})
auctions.head(5)

,date,ref_hash
0,2019-04-23 18:58:00.842116,2564673204772915246
1,2019-04-23 18:58:01.530771,4441121667607578179
2,2019-04-23 18:58:01.767562,7721769811471055264
3,2019-04-23 18:58:02.363468,6416039086842158968
4,2019-04-23 18:58:02.397559,1258642015983312729


In [3]:
auctions=auctions.sort_values(by='date')

In [4]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
auctions_ventana1=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)

In [5]:
fecha_minima=pd.to_datetime('2019-04-19 00:00:00')
fecha_tope=pd.to_datetime('2019-04-22 00:00:00')
auctions_ventana2=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)

In [6]:
fecha_minima=pd.to_datetime('2019-04-20 00:00:00')
fecha_tope=pd.to_datetime('2019-04-23 00:00:00')
auctions_ventana3=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)

In [7]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')
auctions_ventana4=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)

In [8]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')
auctions_ventana7=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)

In [9]:
features = ['sum', 'mean', 'std', 'min', 'max']
ventanas = [auctions_ventana1, auctions_ventana2, auctions_ventana3, auctions_ventana4, auctions_ventana7]
numero_de_ventanas = [1,2,3,4,7]
contador = 0


In [10]:
for ventana in ventanas:
    ventana['time_diff'] = ventana.groupby('ref_hash').agg('diff')/ np.timedelta64(1, 's')
    ventana_aux = ventana.groupby('ref_hash')['time_diff'].agg('sum').to_frame()
    ventana_aux['time_diff_mean'] = ventana.groupby('ref_hash')['time_diff'].agg('mean')
    ventana_aux['time_diff_std'] = ventana.groupby('ref_hash')['time_diff'].agg('std')
    ventana_aux['time_diff_max'] = ventana.groupby('ref_hash')['time_diff'].agg('max')
    ventana_aux['time_diff_min'] = ventana.groupby('ref_hash')['time_diff'].agg('min')
    ventana_aux = ventana_aux.rename(columns = {'time_diff':'time_diff_sum'})
    ventana_aux = ventana_aux.fillna(259200)
    ventana_aux['time_diff_sum'] = (ventana_aux['time_diff_sum'] == 0) * 259200 + ventana_aux['time_diff_sum']
    ventana_aux.to_csv('../xgb/features_timediff_auctions_ventana' + str(numero_de_ventanas[contador]) + '.csv')
    contador = contador + 1